## Totalt nedbørsmengde per år


In [9]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Definerer rotmappen for prosjektet basert på nåværende arbeidskatalog
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Gå ett nivå opp fra 'notebooks'

# Filstier til dataene
oslo_csv = os.path.join(project_root, "data", "processed", "vaerdata_oslo.csv")
tromso_csv = os.path.join(project_root, "data", "processed", "vaerdata_tromso.csv")

# Leser inn dataene
df_oslo = pd.read_csv(oslo_csv)
df_tromso = pd.read_csv(tromso_csv)

# Filtrerer dataene for nedbør (precipitation) og lager eksplisitte kopier
df_oslo_precipitation = df_oslo[
    (df_oslo['elementId'] == 'sum(precipitation_amount P1D)') & 
    (df_oslo['timeOffset'] == 'PT18H')
].copy()
df_tromso_precipitation = df_tromso[
    (df_tromso['elementId'] == 'sum(precipitation_amount P1D)') & 
    (df_tromso['timeOffset'] == 'PT18H')
].copy()

# Konverterer 'referenceTime' til datetime for korrekt gruppering
df_oslo_precipitation['referenceTime'] = pd.to_datetime(df_oslo_precipitation['referenceTime'])
df_tromso_precipitation['referenceTime'] = pd.to_datetime(df_tromso_precipitation['referenceTime'])

# Legger til en kolonne for år
df_oslo_precipitation['year'] = df_oslo_precipitation['referenceTime'].dt.year
df_tromso_precipitation['year'] = df_tromso_precipitation['referenceTime'].dt.year

# Summerer nedbørsmengden per år
annual_oslo_precipitation = df_oslo_precipitation.groupby('year')['value'].sum().reset_index()
annual_tromso_precipitation = df_tromso_precipitation.groupby('year')['value'].sum().reset_index()

# Legger til en kolonne for by
annual_oslo_precipitation['city'] = 'Oslo'
annual_tromso_precipitation['city'] = 'Tromsø'

# Kombinerer dataene for Oslo og Tromsø
annual_precipitation = pd.concat([annual_oslo_precipitation, annual_tromso_precipitation])

# Visualiserer dataene med Seaborn
plt.figure(figsize=(12, 6))
sns.barplot(data=annual_precipitation, x='year', y='value', hue='city', palette='coolwarm')

# Legger til tittel og akseetiketter
plt.title('Total Annual Precipitation for Oslo and Tromsø')
plt.xlabel('Year')
plt.ylabel('Total Precipitation (mm)')
plt.xticks(rotation=45)
plt.legend(title='City')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Viser grafen
plt.tight_layout()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ThomasKonglevoll/Desktop/Miljodataanalyseapplikasjon/src/data/processed/vaerdata_oslo.csv'

## Tempratur forskjell by

In [10]:
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import os
import calendar

# Definerer rotmappen for prosjektet basert på nåværende arbeidskatalog
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Gå ett nivå opp fra 'notebooks'

# Filstier til dataene
oslo_csv = os.path.join(project_root, "data", "processed", "vaerdata_oslo.csv")
tromso_csv = os.path.join(project_root, "data", "processed", "vaerdata_tromso.csv")

# Leser inn dataene
df_oslo = pd.read_csv(oslo_csv)
df_tromso = pd.read_csv(tromso_csv)

# Konverter 'referenceTime' til datetime-format
df_oslo['referenceTime'] = pd.to_datetime(df_oslo['referenceTime'])
df_tromso['referenceTime'] = pd.to_datetime(df_tromso['referenceTime'])

# Legg til kolonner for år, måned og dag
df_oslo['year'] = df_oslo['referenceTime'].dt.year
df_oslo['month'] = df_oslo['referenceTime'].dt.month_name()
df_oslo['day'] = df_oslo['referenceTime'].dt.date
df_tromso['year'] = df_tromso['referenceTime'].dt.year
df_tromso['month'] = df_tromso['referenceTime'].dt.month_name()
df_tromso['day'] = df_tromso['referenceTime'].dt.date

# Filtrer dataene for maks og min temperatur for Oslo
df_oslo_max = df_oslo[df_oslo["elementId"] == "max(air_temperature P1D)"][["day", "value"]].rename(columns={"value": "temp_max"})
df_oslo_min = df_oslo[df_oslo["elementId"] == "min(air_temperature P1D)"][["day", "value"]].rename(columns={"value": "temp_min"})

# Filtrer dataene for maks og min temperatur for Tromsø
df_tromso_max = df_tromso[df_tromso["elementId"] == "max(air_temperature P1D)"][["day", "value"]].rename(columns={"value": "temp_max"})
df_tromso_min = df_tromso[df_tromso["elementId"] == "min(air_temperature P1D)"][["day", "value"]].rename(columns={"value": "temp_min"})

# Slå sammen maks og min temperatur for Oslo
df_oslo_daily = pd.merge(df_oslo_max, df_oslo_min, on="day")
df_oslo_daily["temp_diff"] = df_oslo_daily["temp_max"] - df_oslo_daily["temp_min"]

# Slå sammen maks og min temperatur for Tromsø
df_tromso_daily = pd.merge(df_tromso_max, df_tromso_min, on="day")
df_tromso_daily["temp_diff"] = df_tromso_daily["temp_max"] - df_tromso_daily["temp_min"]

# Legg til år og måned for daglige data
df_oslo_daily["year"] = pd.to_datetime(df_oslo_daily["day"]).dt.year
df_oslo_daily["month"] = pd.to_datetime(df_oslo_daily["day"]).dt.month_name()
df_tromso_daily["year"] = pd.to_datetime(df_tromso_daily["day"]).dt.year
df_tromso_daily["month"] = pd.to_datetime(df_tromso_daily["day"]).dt.month_name()

# Beregn gjennomsnittlig daglig temperaturforskjell per måned og år
df_oslo_avg = df_oslo_daily.groupby(["year", "month"])["temp_diff"].mean().reset_index()
df_tromso_avg = df_tromso_daily.groupby(["year", "month"])["temp_diff"].mean().reset_index()

# Opprett Dash-applikasjonen
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Temperature Difference Comparison"),
    html.Label("City:"),
    dcc.Dropdown(
        id='city-dropdown',
        options=[
            {'label': 'Oslo', 'value': 'Oslo'},
            {'label': 'Tromsø', 'value': 'Tromsø'}
        ],
        value='Oslo'
    ),
    html.Label("Month:"),
    dcc.Dropdown(
        id='month-dropdown',
        options=[{'label': month, 'value': month} for month in calendar.month_name[1:]],
        value='January'
    ),
    html.Label("Start Year:"),
    dcc.Input(id='start-year', type='number', value=2000, min=2000, max=2023),
    html.Label("End Year:"),
    dcc.Input(id='end-year', type='number', value=2023, min=2000, max=2023),
    dcc.Graph(id='temperature-graph')
])

@app.callback(
    Output('temperature-graph', 'figure'),
    Input('city-dropdown', 'value'),
    Input('month-dropdown', 'value'),
    Input('start-year', 'value'),
    Input('end-year', 'value')
)
def update_graph(city, month, start_year, end_year):
    if city == "Oslo":
        df_filtered = df_oslo_avg[(df_oslo_avg['year'] >= start_year) & 
                                  (df_oslo_avg['year'] <= end_year) & 
                                  (df_oslo_avg['month'] == month)]
    elif city == "Tromsø":
        df_filtered = df_tromso_avg[(df_tromso_avg['year'] >= start_year) & 
                                    (df_tromso_avg['year'] <= end_year) & 
                                    (df_tromso_avg['month'] == month)]
    else:
        return px.bar(title="No Data Available")
    
    # Oppdater grafen med farger basert på temperaturforskjell
    fig = px.bar(
        df_filtered,
        x='year',
        y='temp_diff',
        color='temp_diff',  # Farge basert på temperaturforskjell
        color_continuous_scale='YlOrRd',  # Gult til rødt fargeskjema
        title=f'Average Daily Temperature Difference for {month} - {city} ({start_year} to {end_year})',
        labels={"temp_diff": "Temperature Difference (°C)", "year": "Year"}
    )
    return fig

if __name__ == '__main__':
    app.run(debug=True)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ThomasKonglevoll/Desktop/Miljodataanalyseapplikasjon/src/data/processed/vaerdata_oslo.csv'

In [5]:
import os
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import calendar
from functools import lru_cache


class WeatherDataVisualizer:
    """
    Håndterer innlasting av data og beregning av temperaturstatistikk
    for visualisering.
    """

    def __init__(self, data_dir: str):
        self.data_dir = data_dir

    @lru_cache(maxsize=None)
    def _load_city(self, city: str) -> pd.DataFrame:
        path = os.path.join(self.data_dir, f"vaerdata_{city}.csv")
        if not os.path.exists(path):
            raise FileNotFoundError(f"Fant ikke '{path}'")
        
        df = pd.read_csv(path)
        df["referenceTime"] = pd.to_datetime(df["referenceTime"], utc=True, errors="coerce")
        return df

    def compute_monthly_temp_diff(self, city: str) -> pd.DataFrame:
        df = self._load_city(city)
        
        # Filtrer maks og min temperatur
        df_max = df[df["elementId"] == "max(air_temperature P1D)"][["referenceTime", "value"]].rename(columns={"value": "temp_max"})
        df_min = df[df["elementId"] == "min(air_temperature P1D)"][["referenceTime", "value"]].rename(columns={"value": "temp_min"})
        
        # Slå sammen maks og min og beregn differanse
        df_daily = pd.merge(df_max, df_min, on="referenceTime")
        df_daily["temp_diff"] = df_daily["temp_max"] - df_daily["temp_min"]
        df_daily["year"] = df_daily["referenceTime"].dt.year
        df_daily["month"] = df_daily["referenceTime"].dt.month_name()
        
        # Beregn gjennomsnittlig temperaturforskjell
        df_avg = df_daily.groupby(["year", "month"])["temp_diff"].mean().reset_index()
        return df_avg


# Initialiser visualiseringsklasse
data_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "data", "processed"))
visualizer = WeatherDataVisualizer(data_dir)

# Opprett Dash-applikasjonen
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Temperature Difference Comparison"),
    html.Label("City:"),
    dcc.Dropdown(
        id='city-dropdown',
        options=[
            {'label': 'Oslo', 'value': 'oslo'},
            {'label': 'Tromsø', 'value': 'tromso'}
        ],
        value='oslo'
    ),
    html.Label("Month:"),
    dcc.Dropdown(
        id='month-dropdown',
        options=[{'label': month, 'value': month} for month in calendar.month_name[1:]],
        value='January'
    ),
    html.Label("Start Year:"),
    dcc.Input(id='start-year', type='number', value=2000, min=2000, max=2023),
    html.Label("End Year:"),
    dcc.Input(id='end-year', type='number', value=2023, min=2000, max=2023),
    dcc.Graph(id='temperature-graph')
])

@app.callback(
    Output('temperature-graph', 'figure'),
    Input('city-dropdown', 'value'),
    Input('month-dropdown', 'value'),
    Input('start-year', 'value'),
    Input('end-year', 'value')
)
def update_graph(city, month, start_year, end_year):
    df_avg = visualizer.compute_monthly_temp_diff(city)
    df_filtered = df_avg[
        (df_avg["year"] >= start_year) &
        (df_avg["year"] <= end_year) &
        (df_avg["month"] == month)
    ]
    
    fig = px.bar(
        df_filtered,
        x='year',
        y='temp_diff',
        color='temp_diff',
        color_continuous_scale='YlOrRd',
        title=f'Average Daily Temperature Difference for {month} - {city.capitalize()} ({start_year} to {end_year})',
        labels={"temp_diff": "Temperature Difference (°C)", "year": "Year"}
    )
    return fig

if __name__ == '__main__':
    app.run(debug=True)


Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


SystemExit: 1

/Users/ThomasKonglevoll/Desktop/Miljodataanalyseapplikasjon/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3675: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

